### **CONECTANDO COM O GOOGLE SHEETS**

In [ ]:
# %pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

from __future__ import print_function

import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

linhas = 'A1:F15000'

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1Rwicv0cOdAwUl1Z8RV91LmBiuq8G8lYoox6G3yLnPoc'
SAMPLE_RANGE_NAME = f'Página1!{linhas}'



def main():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute()
        
        valores = result['values']
        # print(valores)
        return valores

    except HttpError as err:
        print(err)


if __name__ == '__main__':
    main()

valores = main()
print(valores)
print(len(valores))

### **IMPORTANDO AS TABELAS AUXILIARES**

In [ ]:
import pandas as pd

# PROMOTORA E UF-CIDADE
df_promotora_cidade = pd.read_csv('DADOS BASE\\DimPromotoras.csv')
df_promotora_cidade['Admissao'] = pd.to_datetime(df_promotora_cidade['Admissao'])
df_promotora_cidade.sort_values(by='Admissao', inplace=True, ascending=False)
df_promotora_cidade = df_promotora_cidade[['Promotora', 'Cidade', 'Admissao']]
df_promotora_cidade.rename(columns={'Cidade': 'UF-CIDADE', 'Promotora': 'PROMOTORA'}, inplace=True)

# EXPORTAR PARA EXCEL
# df_promotora_cidade.to_excel("DADOS BASE\\DimPromotoras.xlsx", index=False)
df_promotora_cidade.drop(axis=0, columns=['Admissao'], inplace=True)

# display(df_promotora_cidade.info())

# UF-CIDADE E REGIÕES
df_regioes = pd.read_excel('DADOS BASE\\DimRegioes.xlsx')
df_regioes.dropna(inplace=True)
# display(df_regioes.info())

# PROMOTORAS / UF-CIDADE / REGIÃO
df_promotoras_uf_cidade_regiao = pd.merge(df_promotora_cidade, df_regioes, on='UF-CIDADE')
df_promotoras_uf_cidade_regiao = df_promotoras_uf_cidade_regiao[['PROMOTORA', 'REGIÃO', 'UF-CIDADE']]

# display(df_promotoras_uf_cidade_regiao.head())

# PROMOTORA/VALOR PROMOTORA
df_valor_promotora = pd.read_excel('DADOS BASE\\DimValorPromot.xlsx')
df_valor_promotora.drop(axis=1, columns='UF-CIDADE', inplace=True)
df_valor_promotora['Valor'] = pd.to_numeric(df_valor_promotora['Valor'].replace(',', '.'))
df_valor_promotora



### **DEFININDO O PERÍODO DE RESULTADOS**

In [ ]:
from datetime import datetime, timedelta

numero_semanas_mes = 5
nome_mes = 'JANEIRO'
ano_atual = 2024

data_inicio_date = datetime(2024, 2, 5)
data_fim_date = datetime(2024, 3, 2)

data_inicio_str = datetime.strftime(data_inicio_date, '%d/%m/%Y')
data_fim_str = datetime.strftime(data_fim_date, '%d/%m/%Y')

if numero_semanas_mes == 5:
    semanas = {
        'SEMANA 1': (),
        'SEMANA 2': (),
        'SEMANA 3': (),
        'SEMANA 4': (),
        'SEMANA 5': ()
    }
else:
    semanas = {
    'SEMANA 1': (),
    'SEMANA 2': (),
    'SEMANA 3': (),
    'SEMANA 4': ()
}

data_variavel = data_inicio_date

for semana in range(numero_semanas_mes):
    semanas[f'SEMANA {semana + 1}'] = ((data_variavel + timedelta(5) - timedelta(5), 
                                        data_variavel + timedelta(5), 
                                        f'{(datetime.strftime(data_variavel + timedelta(5) - timedelta(5), "%d/%m"))} A {(datetime.strftime(data_variavel + timedelta(5), "%d/%m"))}'))
    data_variavel = data_variavel + timedelta(7)

### **TRANSFORMANDO OS VALORES EM UM DATAFRAME INICIAL DE BASE**
#### **TRATAMENTOS DE COLUNAS E FILTRO DE DATA E PRODUTO JÁ APLICADOS**
#### **JUNTANDO COM OS DADOS DE REGIÕES E COM O VALOR**

In [ ]:
df_resultado_promotoras = {
    'PROMOTORA': [],
    'DIA': [],
    'LOJA': [],
    'QUANTIDADE': [],
    'PRODUTO': [],
    'PRAÇA': []
}


for cont, linha in enumerate(valores):
    linha = tuple(linha)

    if cont != 0:
        promotora, dia, loja, qtd, produto, praca = linha

        filtro_produto = (produto.strip() == 'ARROZ')        
        filtro_data = (data_inicio_date <= datetime.strptime(dia, '%d/%m/%Y') <= data_fim_date)
        filtro_qtd = (qtd.strip().isnumeric())
        
        if filtro_produto and filtro_data and filtro_qtd:
            df_resultado_promotoras['PROMOTORA'].append(promotora)
            df_resultado_promotoras['DIA'].append(dia)
            df_resultado_promotoras['LOJA'].append(loja)
            df_resultado_promotoras['QUANTIDADE'].append(round(int(qtd) / 6))
            df_resultado_promotoras['PRODUTO'].append(produto)
            df_resultado_promotoras['PRAÇA'].append(praca)

df_resultado_promotoras = pd.DataFrame(df_resultado_promotoras)
df_resultado_promotoras = pd.merge(df_resultado_promotoras, df_promotoras_uf_cidade_regiao, on='PROMOTORA', how='left')
df_resultado_promotoras = pd.merge(df_resultado_promotoras, df_valor_promotora, on='PROMOTORA', how='left')
df_resultado_promotoras.fillna(0, inplace=True)

print(df_resultado_promotoras.info())
display(df_resultado_promotoras.head(8))


### **TOTAL PERIODO E MEDIA DIARIA**

In [ ]:
df_total_periodo_media = pd.DataFrame(df_resultado_promotoras.groupby(['PROMOTORA', 'PRAÇA', 'REGIÃO', 'UF-CIDADE', 'Valor'])['QUANTIDADE'].sum()).reset_index().sort_values(by='PROMOTORA')

df_total_periodo_media['MEDIA DIARIA'] = round(df_total_periodo_media['QUANTIDADE'] / pd.DataFrame(df_resultado_promotoras['PROMOTORA'].value_counts()).reset_index().sort_values(by='PROMOTORA')['count'].reset_index(drop=True))

df_total_periodo_media.rename(columns={'QUANTIDADE': 'TOTAL PERÍODO'}, inplace=True)

df_total_periodo_media[['PROMOTORA', 'TOTAL PERÍODO']].tail()




### **SEMANA 1**

In [ ]:
inicio_semana1_date, fim_semana1_date, periodo_semana1_str = semanas['SEMANA 1']

df_resultado_semana1 = {
    'PROMOTORA': [],
    periodo_semana1_str: [],
}




for i in df_resultado_promotoras.index:
    if inicio_semana1_date <= datetime.strptime(df_resultado_promotoras.loc[i, 'DIA'], '%d/%m/%Y') <= fim_semana1_date:
        df_resultado_semana1['PROMOTORA'].append(df_resultado_promotoras.loc[i, 'PROMOTORA'])
        df_resultado_semana1[periodo_semana1_str].append(df_resultado_promotoras.loc[i, 'QUANTIDADE'])

df_resultado_semana1 = pd.DataFrame(pd.DataFrame(df_resultado_semana1).groupby('PROMOTORA')[periodo_semana1_str].sum()).reset_index()
df_resultado_semana1.head()

In [ ]:
pd.DataFrame(df_resultado_promotoras['PROMOTORA'].value_counts()).reset_index()

### **SEMANA 2**

In [ ]:
inicio_semana2_date, fim_semana2_date, periodo_semana2_str = semanas['SEMANA 2']

df_resultado_semana2 = {
    'PROMOTORA': [],
    periodo_semana2_str: [],
}




for i in df_resultado_promotoras.index:
    if inicio_semana2_date <= datetime.strptime(df_resultado_promotoras.loc[i, 'DIA'], '%d/%m/%Y') <= fim_semana2_date:
        df_resultado_semana2['PROMOTORA'].append(df_resultado_promotoras.loc[i, 'PROMOTORA'])
        df_resultado_semana2[periodo_semana2_str].append(df_resultado_promotoras.loc[i, 'QUANTIDADE'])

df_resultado_semana2 = pd.DataFrame(pd.DataFrame(df_resultado_semana2).groupby('PROMOTORA')[periodo_semana2_str].sum()).reset_index()
df_resultado_semana2.head()

### **SEMANA 3**

In [ ]:
inicio_semana3_date, fim_semana3_date, periodo_semana3_str = semanas['SEMANA 3']

df_resultado_semana3 = {
    'PROMOTORA': [],
    periodo_semana3_str: [],
}




for i in df_resultado_promotoras.index:
    if inicio_semana3_date <= datetime.strptime(df_resultado_promotoras.loc[i, 'DIA'], '%d/%m/%Y') <= fim_semana3_date:
        df_resultado_semana3['PROMOTORA'].append(df_resultado_promotoras.loc[i, 'PROMOTORA'])
        df_resultado_semana3[periodo_semana3_str].append(df_resultado_promotoras.loc[i, 'QUANTIDADE'])

df_resultado_semana3 = pd.DataFrame(pd.DataFrame(df_resultado_semana3).groupby('PROMOTORA')[periodo_semana3_str].sum()).reset_index()
df_resultado_semana3.head()

### **SEMANA 4**

In [ ]:
inicio_semana4_date, fim_semana4_date, periodo_semana4_str = semanas['SEMANA 4']

df_resultado_semana4 = {
    'PROMOTORA': [],
    periodo_semana4_str: [],
}




for i in df_resultado_promotoras.index:
    if inicio_semana4_date <= datetime.strptime(df_resultado_promotoras.loc[i, 'DIA'], '%d/%m/%Y') <= fim_semana4_date:
        df_resultado_semana4['PROMOTORA'].append(df_resultado_promotoras.loc[i, 'PROMOTORA'])
        df_resultado_semana4[periodo_semana4_str].append(df_resultado_promotoras.loc[i, 'QUANTIDADE'])

df_resultado_semana4 = pd.DataFrame(pd.DataFrame(df_resultado_semana4).groupby('PROMOTORA')[periodo_semana4_str].sum()).reset_index()
df_resultado_semana4.head()

### **SEMANA 5**

In [ ]:
if numero_semanas_mes == 5:
    inicio_semana5_date, fim_semana5_date, periodo_semana5_str = semanas['SEMANA 5']

    df_resultado_semana5 = {
        'PROMOTORA': [],
        periodo_semana5_str: [],
    }




    for i in df_resultado_promotoras.index:
        if inicio_semana5_date <= datetime.strptime(df_resultado_promotoras.loc[i, 'DIA'], '%d/%m/%Y') <= fim_semana5_date:
            df_resultado_semana5['PROMOTORA'].append(df_resultado_promotoras.loc[i, 'PROMOTORA'])
            df_resultado_semana5[periodo_semana5_str].append(df_resultado_promotoras.loc[i, 'QUANTIDADE'])

    df_resultado_semana5 = pd.DataFrame(pd.DataFrame(df_resultado_semana5).groupby('PROMOTORA')[periodo_semana5_str].sum()).reset_index()
    df_resultado_semana5.head()

## **JUNTANDO TODAS AS TABELAS E EXPORTANDO PARA EXCEL**

In [ ]:
df_final = pd.merge(df_total_periodo_media, df_resultado_semana1, on='PROMOTORA', how='left')
df_final = pd.merge(df_final, df_resultado_semana2, on='PROMOTORA', how='left')
df_final = pd.merge(df_final, df_resultado_semana3, on='PROMOTORA', how='left')
df_final = pd.merge(df_final, df_resultado_semana4, on='PROMOTORA', how='left')

if numero_semanas_mes == 5:
    df_final = pd.merge(df_final, df_resultado_semana5, on='PROMOTORA', how='left')
df_final = df_final[[
    'PROMOTORA', 
    'REGIÃO', 
    'UF-CIDADE', 
    'PRAÇA', 
    periodo_semana1_str,
    periodo_semana2_str, 
    periodo_semana3_str,
    periodo_semana4_str,
    periodo_semana5_str,
    'TOTAL PERÍODO',
    'MEDIA DIARIA',
    'Valor'
]]
df_final.fillna(0, inplace=True)
df_final.sort_values(by='MEDIA DIARIA', ascending=False, inplace=True)


df_final.to_excel(f'ARQUIVOS PRONTOS\\RESULTADO DE PROMOTORAS - {nome_mes} {ano_atual}.xlsx', index=False)
os.startfile(f'ARQUIVOS PRONTOS\\RESULTADO DE PROMOTORAS - {nome_mes} {ano_atual}.xlsx')